In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from openeye import oechem, oedepict, oeomega, oequacpac, oemedchem
from IPython.display import display
import pandas as pd
import oenotebook as oenb
from openmoltools.openeye import get_charges
from openmoltools import openeye
from torsionfit.qmscan.enumfrags2pdf import *
import networkx as nx
from torsionfit.qmscan.enumfrags2pdf import main as vis_wrapper

In [2]:
f = open('../torsion_drive/biphenyl/4phepyr.smi', 'r')
smiles = {}
for line in f:
    smile, name = line.rstrip().split(' ')
    smiles[name] = smile
f.close()


In [3]:
# Charge molecule. This takes some time
mol = oechem.OEGraphMol()
simple_oemol = openeye.smiles_to_oemol(test_simple)
charged = get_charges(simple_oemol, keep_confs=1)

In [3]:
def oemol_to_graph(oemol):
    """
    Convert molecule to networkX graph and add WibergBondOrder as edge weight
    
    parameters
    ----------
    oemol: charged openeye OEMolGraph
    
    Returns
    ------
    G: NetworkX Graph
    """
    G = nx.Graph()
    for atom in oemol.GetAtoms():
        G.add_node(atom.GetIdx(), name=atom.GetName())
    for bond in oemol.GetBonds():
        G.add_edge(bond.GetBgnIdx(), bond.GetEndIdx(), weight=bond.GetData("WibergBondOrder"), index=bond.GetIdx())
    return G

In [4]:
bondOrderThreshold = 1.3
def frag_graph(G, bondOrderThreshold=bondOrderThreshold):
    """
    Fragment all bonds with Wiberg Bond Order less than threshold
    
    Parameters
    ----------
    G: NetworkX graph
    bondOrderThreshold: int
        thershold for fragmenting graph
    
    Returns
    -------
    subgraphs: list of subgraphs
    """
    ebunch = []
    for node in G.edge:
        if G.degree(node) <= 1:
            continue
        for node2 in G.edge[node]:
            if G.edge[node][node2]['weight'] < bond_order_threshold and G.degree(node2) >1:
                ebunch.append((node, node2))
    # Cut molecule            
    G.remove_edges_from(ebunch)
    # Generate fragments
    subgraphs = list(nx.connected_component_subgraphs(G))
    return subgraphs

In [5]:
def subgraph_to_atombondset(graph, subgraph, oemol):
    """
    Build Openeye AtomBondSet from subrgaphs for enumerating fragments recipe
    
    Parameters
    ----------
    graph: NetworkX graph
    subgraph: NetworkX subgraph
    oemol: Openeye OEMolGraph
    
    Returns
    ------
    atomBondSet: Openeye oechem atomBondSet
    """
    # Build openeye atombondset from subgraphs
    atomBondSet = oechem.OEAtomBondSet()
    for node in subgraph.node:
        atomBondSet.AddAtom(oemol.GetAtom(oechem.OEHasAtomIdx(node)))
    for node1, node2 in subgraph.edges():
        index = graph.edge[node1][node2]['index']   
        atomBondSet.AddBond(oemol.GetBond(oechem.OEHasBondIdx(index)))
    return atomBondSet

In [6]:
for name in smiles:
    mol = oechem.OEGraphMol()
    oemol = openeye.smiles_to_oemol(smiles[name])
    charged = get_charges(oemol, keep_confs=1)
    
    G = oemol_to_graph(charged)
    subgraphs = frag_graph(G)
    frags = []
    for subgraph in subgraphs:
        frags.append(subgraph_to_atombondset(G, subgraph, charged))
        
    oname = '{}.pdf'.format(name)
    max_rotors = 2
    min_rotors = 1
    
    itf = oechem.OEInterface()
    oedepict.OEPrepareDepiction(charged)
    # Initialize multi page display
    ropts = oedepict.OEReportOptions()
    oedepict.OESetupReportOptions(ropts, itf)
    ropts.SetFooterHeight(25.0)
    ropts.SetHeaderHeight(ropts.GetPageHeight() / 4.0)
    report = oedepict.OEReport(ropts)
    
    # setup decpiction options
    opts = oedepict.OE2DMolDisplayOptions()
    oedepict.OESetup2DMolDisplayOptions(opts, itf)
    cellwidth, cellheight = report.GetCellWidth(), report.GetCellHeight()
    opts.SetDimensions(cellwidth, cellheight, oedepict.OEScale_AutoScale)
    opts.SetTitleLocation(oedepict.OETitleLocation_Hidden)
    opts.SetAtomColorStyle(oedepict.OEAtomColorStyle_WhiteMonochrome)
    opts.SetAtomLabelFontScale(1.2)

    DepictMoleculeWithFragmentCombinations(report, charged, frags, opts, max_rotors, min_rotors)
    
    oedepict.OEWriteReport(oname, report)
    
    